In [1]:
import pandas as pd
from projects.ccs.rooftop_solar_project import RooftopSolarProject



In [30]:
all_df=pd.read_csv('/Volumes/Samsung_T5/data/ccs/6kw_solar_npc_states_3tilts_9azimuths.csv',index_col=[0])

In [11]:
time_varying_carbon_intensity_df=pd.read_csv('/Volumes/Samsung_T5/data/ccs/carbon_intensity_forecasts_for_us_grid - us_fossil_fuel_carbon_intensity_projections.csv')

In [24]:
all_df

,kwh_per_yr,lat,lon,state,tilt,azimuth,kw,region,tco2_per_kwh,installation_cost_scenario,installation_cost_usd,usd_per_kwh,pv_solar_usd_per_tco2,pv_expense_usd_per_tco2,total_co2,tco2_per_yr,npv
0,8204.322296,32.318231,-86.902298,AL,18,105.00,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,115.003355,151.472419,97.047371,NaN,-3539.226755
1,8511.235356,32.318231,-86.902298,AL,18,123.75,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,115.003355,146.010361,100.677787,NaN,-3121.716697
2,8753.698517,32.318231,-86.902298,AL,18,142.50,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,115.003355,141.966112,103.545838,NaN,-2791.881246
3,8913.110541,32.318231,-86.902298,AL,18,161.25,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,115.003355,139.427032,105.431492,NaN,-2575.024644
4,8968.018854,32.318231,-86.902298,AL,18,180.00,6,gulf_coast,0.000308,low_install_usd,14700.0,0.1587,115.003355,138.573365,106.080992,NaN,-2500.329963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,8581.146036,29.760400,-95.369800,TXE,36,217.50,6,gulf_coast,0.000344,high_install_usd,17220.0,0.1490,107.974165,169.647238,101.504747,NaN,-6260.109718
1562,8242.116886,29.760400,-95.369800,TXE,36,236.25,6,gulf_coast,0.000344,high_install_usd,17220.0,0.1490,107.974165,176.625465,97.494436,NaN,-6693.119746
1563,8242.116886,29.760400,-95.369800,TXE,36,236.25,6,gulf_coast,0.000344,high_install_usd,17220.0,0.1490,107.974165,176.625465,97.494436,NaN,-6693.119746
1564,7808.018567,29.760400,-95.369800,TXE,36,255.00,6,gulf_coast,0.000344,high_install_usd,17220.0,0.1490,107.974165,186.445218,92.359569,NaN,-7247.552677


In [27]:
shared_params={}
shared_params['inflation_rate']=0.025
shared_params['discount_rate']=0.12
shared_params['tco2_per_kwh']=list(time_varying_carbon_intensity_df['carbon intensity'])
shared_params['project_length_yrs']=25


In [32]:
for i,row in all_df.loc[all_df.installation_cost_scenario=='low_install_usd'].iterrows():
       s=RooftopSolarProject(row[['kwh_per_yr', 'lat', 'lon', 'state', 'tilt', 'azimuth',
       'kw', 'region',
       'installation_cost_usd', 'usd_per_kwh']].to_dict() | shared_params)
       all_df.at[i,'pv_solar_usd_per_tco2']=s.pv_solar_usd_per_tco2
       all_df.at[i,'pv_expense_usd_per_tco2']=s.pv_expense_usd_per_tco2
       all_df.at[i,'npv']=s.npv
       all_df.at[i,'total_co2']=s.total_tco2

       


In [33]:
all_df.loc[all_df.installation_cost_scenario=='low_install_usd'].describe()

,kwh_per_yr,lat,lon,tilt,azimuth,kw,tco2_per_kwh,installation_cost_usd,usd_per_kwh,pv_solar_usd_per_tco2,pv_expense_usd_per_tco2,npv,total_co2
count,783.000000,783.000000,783.000000,783.000000,783.000000,783.0,783.000000,783.000000,783.000000,783.000000,783.000000,783.000000,783.000000
mean,8644.477646,38.063405,-95.826267,27.000000,180.000000,6.0,0.000407,16092.965517,0.168183,146.931554,159.542489,-973.794532,102.253885
std,954.374736,5.029747,12.929298,7.353166,48.443236,0.0,0.000189,1431.437550,0.078755,68.803520,25.194094,7703.259376,11.289118
min,6418.019223,27.664827,-122.419400,18.000000,105.000000,6.0,0.000154,14040.000000,0.104400,91.208245,119.142949,-8084.977891,75.917531
25%,7954.570029,35.007752,-105.032363,18.000000,142.500000,6.0,0.000288,14760.000000,0.123800,108.156903,142.011357,-4744.915101,94.093098
50%,8589.052127,38.597626,-95.369800,27.000000,180.000000,6.0,0.000352,15840.000000,0.145300,126.940210,152.933704,-3317.090358,101.598267
75%,9278.622036,41.203322,-85.602364,36.000000,217.500000,6.0,0.000478,17100.000000,0.161000,140.656393,169.109718,-1479.416887,109.755058
max,11462.085450,47.551493,-74.217933,36.000000,255.000000,6.0,0.000870,20220.000000,0.413000,360.814225,251.081928,25961.389647,135.582832
